# Deep learning par la pratique - manipulation des tenseurs

Dans ce notebook vous allez vous familiariser avec la manipulation des tenseurs avec tensorflow



In [ ]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

## operations de base

### Création des tenseurs

On crée des tenseurs de rang 0 à 5 à partir d'objets numpy

In [ ]:
# Rank 0 Tensor (Vector)
tensor_rank_0 = tf.constant(22)
print("Rank 0 Tensor:", tensor_rank_0)

In [ ]:
# Rank 1 Tensor (Vector)
tensor_rank_1 = tf.constant([1, 2, 3, 4, 5])
print("Rank 1 Tensor:", tensor_rank_1)

In [ ]:
# Rank 2 Tensor (Matrix)
tensor_rank_2 = tf.constant([[1, 2, 3], [4, 5, 6]])
print("Rank 2 Tensor:", tensor_rank_2)


#### A vous

Chargez un fichier csv (Iris) dans un tenseur
en utilisant la fonction
`tf.convert_to_tensor()`

- qu'est ce qui se passe et pourquoi ?


In [ ]:
# chargez un fichier csv (Iris) dans un tenseur
# en utilisant la fonction
# tf.convert_to_tensor(data)

# load iris.csv
import pandas as pd

url = "https://raw.githubusercontent.com/SkatAI/skatai_deeplearning/master/data/iris_dataset.csv"
data = pd.read_csv(url)
print(data.head())
# votre code



In [ ]:
# charger une image
import requests

# lire l'image
image_url = 'https://github.com/SkatAI/skatai_deeplearning/blob/master/img/etretat.jpg?raw=true'
response = requests.get(image_url)
image_data = response.content


In [ ]:
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt


image = Image.open(BytesIO(image_data))
plt.imshow(image)
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
# charger dans un tenseur

# 1) garder les couleurs
image = tf.image.decode_jpeg(image_data, channels=3)
# Print the shape of the tensor
print("Tensor shape:", image.shape)


In [ ]:
# forcer en noir et blanc
# quelles est la shape du tenseur

# 1) garder les couleurs
image = tf.image.decode_jpeg(image_data, channels=1)
# Print the shape of the tensor
print("Tensor shape:", image.shape)



In [ ]:
# Display the image using matplotlib
plt.imshow(image.numpy().squeeze(), cmap='gray')
plt.axis('off')  # Hide axes
plt.show()

Que fait squeeze ?


In [ ]:
# que fait squeeze ?


les valeurs dans `image` sont des int entre 0 et 255

pourr la manipuler on va les normaliser avec

`image = tf.cast(image, tf.float32) / 255.0`


In [ ]:
image = tf.cast(image, tf.float32) / 255.0


# Manipuler des images

## pixelisation / thresholding

avec la fonction `tf.where` on peut mettre a zero les pixels au dessus d'un seuil

`image_seuil = tf.where(image > threshold, 1.0, 0.0)`


In [ ]:
threshold = 0.5
image_seuil = tf.where(image > threshold, 1.0, 0.0)


In [ ]:
# Pour afficher l'image il faut la reconvertir en Int avec des valeurs de 0 a 255
image_seuil = tf.cast(image_seuil * 255, tf.uint8)

In [ ]:
plt.imshow(image_seuil, cmap='gray')


## Melanger 2 images

Il faut d'abord qu'elles soient à la même dimension

on utilise la fonction `tf.reshape`



In [ ]:
# loader les 2 images
etretat = requests.get('https://github.com/SkatAI/skatai_deeplearning/blob/master/img/etretat.jpg?raw=true').content

cerisier = requests.get('https://github.com/SkatAI/skatai_deeplearning/blob/master/img/cerisier.jpg?raw=true').content

In [ ]:
plt.imshow(Image.open(BytesIO(etretat)))


In [ ]:
plt.imshow(Image.open(BytesIO(cerisier)))

In [ ]:
# loader les 2 images dans un tenseur
tf_etretat = tf.image.decode_jpeg(etretat, channels=1)
print(tf_etretat.shape)
tf_cerisier = tf.image.decode_jpeg(cerisier, channels=1)
print(tf_cerisier.shape)

In [ ]:
tf.shape(tf_etretat)[:2]

In [ ]:
# mettre aux memes dimension
target_shape = tf.shape(tf_etretat)[:2]
tf_cerisier_resized = tf.image.resize(tf_cerisier,
            target_shape,
            method=tf.image.ResizeMethod.BILINEAR
)


In [ ]:
print(tf.shape(tf_cerisier_resized))

In [ ]:
plt.imshow(tf_cerisier_resized.numpy().squeeze(), cmap='gray')


## fusionner les 2 images !

In [ ]:
# cast to float
tf_etretat = tf.cast(tf_etretat, tf.float32)
tf_cerisier_resized = tf.cast(tf_cerisier_resized, tf.float32)


In [ ]:
blend = tf.clip_by_value(tf_etretat * 0.5 + tf_cerisier_resized * 0.5, 0, 255)

Que fait la fonction `clip_by_value()` ?

In [ ]:
# cast back to int
blend = tf.cast(blend, tf.uint8)

In [ ]:
# afficher
plt.imshow(blend.numpy().squeeze(), cmap = 'gray')


# filtrage par

on multiplie l'image par un filtre tenseur

ici on filtre le rouge en mettant a > 1 et en réduisant  le bleu et le vert

In [ ]:
etretat = requests.get('https://github.com/SkatAI/skatai_deeplearning/blob/master/img/etretat.jpg?raw=true')
tf_etretat = tf.image.decode_jpeg(etretat.content, channels=3)
print(tf_etretat.shape)
tf_etretat = tf.cast(tf_etretat, tf.float32)
tf_etretat


In [ ]:
red_filter = tf.constant([1.5, 0.5, 0.5], shape=[1, 1, 3])
etretat_rouge = tf.clip_by_value(tf_etretat * red_filter, 0, 255)
etretat_rouge_uint8 = tf.cast(etretat_rouge, tf.uint8)

In [ ]:

plt.imshow(etretat_rouge_uint8.numpy().squeeze())
